<a href="https://colab.research.google.com/github/Kolo-Naukowe-Axion/QC1/blob/dnn/dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Banknote Authentication DNN Developed a binary classification model using PyTorch to detect counterfeit banknotes based on four mathematical image parameters (Variance, Skewness, Kurtosis, Entropy). The solution is based on the UCI Banknote Authentication Data Set, containing 1372 records. Prior Exploratory Data Analysis (EDA) revealed that the dataset is well-balanced (55% authentic / 45% counterfeit) and confirmed a clear separation between authentic and counterfeit classes. This strong linear separability allowed the neural network, combined with StandardScaler normalization, to easily define decision boundaries and achieve near-perfect accuracy on the test set. For benchmarking purposes, a Logistic Regression model was also trained, reaching ~98% accuracy, which further corroborates the high quality and distinct separation of the dataset features.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
# taking data from the UCI Machine Learning repository
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


Taking data, conversion to numpy

In [4]:
banknote_authentication = fetch_ucirepo(id=267)

X_raw = banknote_authentication.data.features
y_raw = banknote_authentication.data.targets

X = X_raw.values
y = y_raw.values.ravel()

Test and train split, 80% train and 20% test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# stadarization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

print(f"Training Tensor Shape: {X_train_tensor.shape}") # 4 different parameters for one banknote: variance	skewness	kurtosis	entropy
print(f"Testing Tensor Shape: {X_test_tensor.shape}")

Training Tensor Shape: torch.Size([1097, 4])
Testing Tensor Shape: torch.Size([275, 4])


In [6]:
# building neural network feed-forward with 3 hidden layers

class BanknoteDNN(nn.Module):
    def __init__(self):
        super(BanknoteDNN, self).__init__()
        self.layer1 = nn.Linear(4, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.sigmoid(self.output(x)) # sigmoid activation, as we want the probability (0 - fake money, 1 - real money)
        return x

model = BanknoteDNN().to(device)

In [7]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
print("\nStarting training...")

for epoch in range(epochs):
    model.train()

    outputs = model(X_train_tensor)

    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        test_loss = criterion(test_outputs, y_test_tensor)
        predicted = (test_outputs > 0.5).float()
        correct = (predicted == y_test_tensor).float().sum()
        accuracy = correct / y_test_tensor.shape[0]

    if epoch%10==0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f} | Test Acc: {accuracy:.4f}")


Starting training...
Epoch 1/100 | Train Loss: 0.7131 | Test Loss: 0.7050 | Test Acc: 0.4618
Epoch 11/100 | Train Loss: 0.6882 | Test Loss: 0.6832 | Test Acc: 0.4618
Epoch 21/100 | Train Loss: 0.6539 | Test Loss: 0.6511 | Test Acc: 0.8145
Epoch 31/100 | Train Loss: 0.5962 | Test Loss: 0.5948 | Test Acc: 0.8545
Epoch 41/100 | Train Loss: 0.5128 | Test Loss: 0.5125 | Test Acc: 0.8545
Epoch 51/100 | Train Loss: 0.4015 | Test Loss: 0.4037 | Test Acc: 0.8800
Epoch 61/100 | Train Loss: 0.2808 | Test Loss: 0.2873 | Test Acc: 0.9055
Epoch 71/100 | Train Loss: 0.1840 | Test Loss: 0.1851 | Test Acc: 0.9382
Epoch 81/100 | Train Loss: 0.1147 | Test Loss: 0.1136 | Test Acc: 0.9745
Epoch 91/100 | Train Loss: 0.0824 | Test Loss: 0.0713 | Test Acc: 0.9818


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(f"Simple model accuracy: {accuracy_score(y_test, preds)*100:.2f}%")

Simple model accuracy: 97.82%
